In [13]:
import requests
from pymongo import MongoClient
from bson.objectid import ObjectId
import re
from bs4 import BeautifulSoup

# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.

years = ["2015", "2016", "2017"]
months = ["jan"]
dates = [str(n) for n in xrange(14,20)]

# count of number of links
i = 0

# Initiating mongoDB. Using pymongo to connect the database
client = MongoClient()
db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database

result = db.urls.delete_many({}) # A fresh start to the DB table -> removing all entries 

for month in months:
    for date in dates:
        root_url = "https://www.theguardian.com/commentisfree/2016/" + month + "/" + date 
        
        # This one contains all the URLs. Soup extracts those.
        list_of_urls = requests.get(root_url).text
        soup = BeautifulSoup(list_of_urls, "html.parser")

        # the .findAll method from re finds the _content_ that has 
        # html tag - 'a', attribute - 'href' and 
        # the pattern that begins with the root_url variable
        for link in soup.findAll('a', attrs={'href': re.compile(root_url)}):

            # urls is the name of the mongo DataBase. insert_one is the method 
            urls.insert_one({str(i) : link.get('href')})
            i += 1
print "Total number of links stored is ", i

Total number of links stored is  248


In [14]:
i = 0
for d in urls.find():
    i += 1
print i

248


In [15]:
url = "https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"

text = requests.get(url).text
soup = BeautifulSoup(text, "html.parser")
# print soup.prettify()

The one below is a github code for downloading comments. Do not use this; instead use a modified version of this in the cell below this one.

In [ ]:
import urllib2
import re
import time
import sys

# This script will download all comments from a Guardian (theguardian.com) article into a single .html file
# Use 'File ID' to easily identify your file later (this could also be something like a Citavi identifier)
# Ingo Kleiber <ikleiber@googlemail.com>
# https://gist.github.com/IngoKl/3ebd968ef3c6da406638

print """\
   _____                     _ _             _____  _      
  / ____|                   | (_)           |  __ \| |     
 | |  __ _   _  __ _ _ __ __| |_  __ _ _ __ | |  | | |     
 | | |_ | | | |/ _` | '__/ _` | |/ _` | '_ \| |  | | |     
 | |__| | |_| | (_| | | | (_| | | (_| | | | | |__| | |____ 
  \_____|\__,_|\__,_|_|  \__,_|_|\__,_|_| |_|_____/|______|
"""                                                         

print 'Guardian Comment Downloader v.1.1 - Ingo Kleiber (05.01.2015)\n'
guardianLink = raw_input ('> Guadian Link: ')
fileId = raw_input('> File ID: ')

#Figuring out the PageId
pageResponse = urllib2.urlopen(guardianLink)
commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
# print "Comments Match = ", commentsMatch()
if commentsMatch:
    pageId = commentsMatch.group(1)
    print '[+] pageId has been retrieved ('+pageId+')'
else:
    sys.exit('[-] Could not retrieve pageId!')

#Retrieving comments
downloadCount = 1
downloadError = 0

startTime = time.time()
fileName = 'GuardianDownload_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
f = open(fileName, 'a')


print '[~] Start downloading pages ...'
while downloadError == 0:
    try:
        response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
        html = response.read()
        f.write(html)
        if downloadCount == 1:
            soup = BeautifulSoup(html, 'html.parser')
        print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
        downloadCount = downloadCount + 1
        
    except:
        downloadError = 1

stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
f.close() 

In [1]:
import urllib2
import re
import time
import sys

# This script will download all comments from a Guardian (theguardian.com) article into a single .html file
# Use 'File ID' to easily identify your file later (this could also be something like a Citavi identifier)
# Ingo Kleiber <ikleiber@googlemail.com>
# https://gist.github.com/IngoKl/3ebd968ef3c6da406638

def download_comments(url, filename):

    guardianLink = url
    fileId = filename

    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    startTime = time.time()
    fileName = 'GuardianComments_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
    f = open(fileName, 'a')


    print '[~] Start downloading pages ...'
    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
            html = response.read()
#             f.write(html)
            if downloadCount == 1:
                soup = BeautifulSoup(html, 'html.parser')
            print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    stopTime = time.time()
    runningTime = (stopTime - startTime)/60;
    print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
    f.close() 

In [ ]:
download_comments(url, filename)

In [ ]:
# soup = BeautifulSoup(html, 'html.parser')

In [17]:
# print soup.prettify()

In [ ]:
soup.comment_body

In [ ]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})
replies = soup.findAll("span", {"class" : "d-comment__reply-to-author"})
for user, comment, recommend, reply in zip(users, comments, recommends, replies):
    print user.text, comment.text, recommend.text, reply.text


a class="social__action social-icon-wrapper" data-link-name="social-comment : facebook" href="https://www.facebook.com/dialog/share?app_id=180444840287&amp;href=https%3A%2F%2Fdiscussion.theguardian.com%2Fcomment-permalink%2F103303986&amp;quote=alexandrerizvin%20commented%3A%20%22My%20fantasy%20of%20freedom%20and%20independence%20has%20been%20to%20have%20no%20dependence%20on%20cars.%22" target="_blank">

In [ ]:
# import requests
url = "https://www.theguardian.com/us/commentisfree"

list_of_urls = requests.get(url).text

# from bs4 import BeautifulSoup 
soup2 = BeautifulSoup(list_of_urls, "html.parser")

# soup2.prettify()

import re
i = 0
for link in soup2.findAll('a', attrs={'href': re.compile("https://www.theguardian.com/commentisfree/2017")}):
    print link.get('href')
    i += 1
print i